In [1]:
import os


In [2]:
%pwd

'c:\\Users\\PASCAL\\flight_price_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\PASCAL\\flight_price_prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [6]:
from src.flightprice.constants import *
from src.flightprice.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        root_dir=config.root_dir,
        source_URL=config.source_URL,
        local_data_file=config.local_data_file,
        unzip_dir=config.unzip_dir
        
    )

        return data_ingestion_config
 

In [9]:
import os
import urllib.request as request
import zipfile
from src.flightprice.logger import logging
from src.flightprice.utils.common import get_size
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
from src.flightprice.logger import logging
from sklearn.model_selection import train_test_split
import pandas as pd
import requests
import urllib
import sys

In [10]:


class DataIngestion:
    def __init__(self, data_ingestion_config):
        self.config = data_ingestion_config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            print(f"{filename} downloaded with the following info:\n{headers}")
        else:
            print(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    def ingest_data(self):
        logging.info("Ingestion of the data is initiated")

        raw_data_path = os.path.join(self.config.unzip_dir, "raw_data.csv")
        train_data_path = os.path.join(self.config.unzip_dir, "train_data.csv")
        test_data_path = os.path.join(self.config.unzip_dir, "test_data.csv")

        data_files = os.listdir(self.config.unzip_dir)
        csv_files = [file_name for file_name in data_files if file_name.endswith(".csv")]

        if len(csv_files) == 0:
            raise ValueError("No CSV files found in the unzip directory.")

        df = None
        for file_name in csv_files:
            file_path = os.path.join(self.config.unzip_dir, file_name)
            if df is None:
                df = pd.read_csv(file_path)
            else:
                df = pd.concat([df, pd.read_csv(file_path)], ignore_index=True)

        # Save the raw data
        df.to_csv(raw_data_path, index=False, header=True)

        # Train test split
        train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

        # Save the train and test data
        train_set.to_csv(train_data_path, index=False, header=True)
        test_set.to_csv(test_data_path, index=False, header=True)

        logging.info("Ingestion of the data is completed")

        return train_data_path, test_data_path



In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    train_data_path, test_data_path = data_ingestion.ingest_data()
except Exception as e:
    raise e

[2023-07-16 03:56:30,821: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-16 03:56:30,824: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-16 03:56:30,826: INFO: common: created directory at: artifacts]
[2023-07-16 03:56:30,829: INFO: common: created directory at: artifacts/data_ingestion]


artifacts/data_ingestion/data.zip downloaded with the following info:
Connection: close
Content-Length: 117805
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dc685b47036210fd4b4ff46f7f0ddc35bcbd54143e11c59cdbd4fe3d47169b13"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8D38:FA0B:1314BEC:13E2875:64B3CCDD
Accept-Ranges: bytes
Date: Sun, 16 Jul 2023 10:56:29 GMT
Via: 1.1 varnish
X-Served-By: cache-ams21038-AMS
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1689504990.640669,VS0,VE186
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 7df499c4ff52cb81e692423f048e5d744c5d16b6
Expires: Sun, 16 Jul 2023 11:01:29 GMT
Source-Age: 0


[2023-07-16 03:56:32,927: INFO: 1643098587: Ingestion of the data is initiated]
[2023